In [1]:
import sys
import PyQt5
from PyQt5 import QtGui
from PyQt5 import QtCore

from PyQt5.QtGui import * 
from PyQt5.QtWidgets import  QApplication, QWidget, QPushButton, QFileDialog, QVBoxLayout, QMessageBox, QLineEdit, QFormLayout,QLabel, QGridLayout, QCheckBox, QSlider
from PyQt5.QtCore import Qt, QTimer, QObject, QThread, pyqtSignal
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg
import scipy.stats as stats
import time
import datetime
from scipy import optimize
from scipy.interpolate import interp2d
import lzma
        


In [2]:
class BeamGui:
    HardCurrentLimit=25
    HardCurrentStepLimit=20
    comdict={'current': 1}
    recordtimes=[]
    currents=[]
    vacuums=[]
    
    CurrentNow = 1 
    def __init__(self):

            self.LastTimeString=datetime.datetime.now().strftime('%m/%d, %H:%M:%S')

            # PyQt setup stuff
            self.app = QApplication([])
            self.window = QWidget()
            self.layout= QGridLayout()

            # Initiate all the GUI frames and make the layout
            #self.SetupCameraThread()
            self.SetupActionsFrame()
            self.SetupFigFrame()
            self.SetupMonitoringFrame()
            self.SetupArduinoThread()
            
            self.TimeOfLastStep=time.time()
            self.FirstTime=time.time()
            
            self.layout.addLayout(self.MonitorLayout,1,0)
            self.layout.addLayout(self.ActionsLayout,3,0)


            self.layout.addLayout(self.FigLayout,1,2,3,1)
            


            #lets go!
            self.window.setLayout(self.layout)
            self.window.show()
            self.app.exec()


    def SetupFigFrame(self):
            self.FigLayout=QVBoxLayout()
            self.fig2= plt.figure(figsize=(5,5),dpi=150)
            #self.fig2.add_axes(rect=[0.1,0,1,1])

            gs = self.fig2.add_gridspec(2, hspace=0)
            gs.update(left=0.2)
            self.ax2 = gs.subplots(sharex=True, sharey=False)

            self.CanvasObj2=FigureCanvasQTAgg(self.fig2)
            #self.MakePosPlots()

            #self.CanvasObj.mpl_connect("button_release_event", self.on_map_press)

            #self.CanvasObj2.mpl_connect("button_press_event", self.on_zplot_press)
            #self.CanvasObj2.mpl_connect("button_release_event", self.on_zplot_release)

            self.FigLayout.addWidget(self.CanvasObj2)
            
    def SetupActionsFrame(self):
            self.ActionsLayout= QGridLayout()

            self.SetCurrent=QPushButton('Set')
            self.SetCurrent.clicked.connect(self.on_setcurrent_click)
            self.CurrentVal=QLineEdit(text=str(1))



            self.Climbing=QCheckBox("Climb current?",checked=False)

            self.CurrentMax=QLineEdit(text=str(100))
            self.ClimbBy=QLineEdit(text=str(5))
            self.TryEvery=QLineEdit(text=str(300))
            self.VacStepThresh=QLineEdit(text=str(2e-5))


            self.LimVac=QCheckBox("Limit Vacuum?",checked=False)

            self.Limit=QLineEdit(text=str(8e-5))
            self.Backoff=QLineEdit(text=str(5))


            self.ActionsLayout.addWidget(QLabel("Set current to:"),1,0,1,3)

            self.ActionsLayout.addWidget(self.CurrentVal,2,0,1,2)
            self.ActionsLayout.addWidget(self.SetCurrent,2,2,1,2)


#            self.ActionsLayout.addWidget(QLabel("Limit vaccum:"),3,0,1,3)
            
            self.ActionsLayout.addWidget(QLabel("  Limit vacuum"),4,0,1,1)
            self.ActionsLayout.addWidget(QLabel("  Max"),4,1,1,1)
            self.ActionsLayout.addWidget(QLabel("  Step back by"),4,2,1,1)

            self.ActionsLayout.addWidget(self.LimVac,5,0,1,1)
            self.ActionsLayout.addWidget(self.Limit,5,1,1,1)
            self.ActionsLayout.addWidget(self.Backoff,5,2,1,1)


#            self.ActionsLayout.addWidget(QLabel("Climb Function:"),6,0,1,3)
 
            self.ActionsLayout.addWidget(self.Climbing,7,1,1,3)  

            self.ActionsLayout.addWidget(QLabel("  Step each (s)"),8,0,1,1)
            self.ActionsLayout.addWidget(QLabel("  by (A)"),8,1,1,1)
            self.ActionsLayout.addWidget(self.TryEvery,9,0,1,1)
            self.ActionsLayout.addWidget(self.ClimbBy,9,1,1,1)
            self.ActionsLayout.addWidget(QLabel("  up to (A)"),10,0,1,1)
            self.ActionsLayout.addWidget(QLabel("  when below (Torr)"),10,1,1,1)
            self.ActionsLayout.addWidget(self.VacStepThresh,11,0,1,1)  
            self.ActionsLayout.addWidget(self.CurrentMax,11,1,1,1)  

            

    def SetupMonitoringFrame(self):
            self.MonitorLayout= QGridLayout()

            self.VacuumQuality=QLineEdit(text=str(5e-5)+"Torr")
            self.VacuumQuality.setFont(QFont('Courier', 20,700))  
            
            self.VacuumQuality.setStyleSheet(
                """QLineEdit { background-color: darkGreen; color: white }""")
            self.VacuumQuality.setAlignment(QtCore.Qt.AlignCenter) 
            
            self.Current=QLineEdit(text="11 A")
            self.Current.setFont(QFont('Courier', 20,700))    
            self.Current.setStyleSheet(
                """QLineEdit { background-color: darkGreen; color: white }""")
            self.Current.setAlignment(QtCore.Qt.AlignCenter) 

            self.LastUpdate=QLabel("Last update: "+str(self.LastTimeString))
            
            self.MonitorLayout.addWidget(QLabel("Set current:"),1,0,1,3)
            self.MonitorLayout.addWidget(self.Current,2,0,1,3)

            self.MonitorLayout.addWidget(QLabel("Vacuum:"),4,0,1,3)
            self.MonitorLayout.addWidget(self.VacuumQuality,5,0,1,3)

            self.MonitorLayout.addWidget(self.LastUpdate,7,0,1,3)

            
    # This starts the thread running for the camera acquisitions
    def SetupArduinoThread(self):
        self.thread = QThread()
        self.worker=DataStream(self.comdict)
        self.worker.moveToThread(self.thread)
        self.thread.started.connect(self.worker.run)
        self.worker.vacuumdat.connect(self.UpdateData)
        self.thread.start()            

        

    def UpdateData(self,data):
        self.VacuumNow=data
        self.VacuumQuality.setText(str('%.2E' % self.VacuumNow)+str(' Torr'))

        self.Current.setText(str('%.2F' % self.CurrentNow)+str(' A'))

        self.LastTimeString=datetime.datetime.now().strftime('%m/%d, %H:%M:%S')
        self.LastUpdate.setText("Last update: "+str(self.LastTimeString))
        
        if(self.LimVac.isChecked()==True):
            if(self.VacuumNow>float(self.Limit.text())):
                if((time.time()-self.TimeOfLastStep)>float(self.TryEvery.text())):
                    self.CurrentVal.setText(str(max(float(self.CurrentVal.text())-float(self.Backoff.text()),0)))
                    self.on_setcurrent_click()
                
        if(self.Climbing.isChecked()==True):
            if((time.time()-self.TimeOfLastStep)>float(self.TryEvery.text())):
                
                if(self.VacuumNow<float(self.VacStepThresh.text())):

                    if(self.CurrentNow< min(float(self.CurrentMax.text()),self.HardCurrentLimit)):
                        self.CurrentVal.setText(str(self.CurrentNow+min(float(self.ClimbBy.text()),self.HardCurrentStepLimit)))
                        self.on_setcurrent_click()

                else:
                    print("not raising I, vacuum check failed")
        self.recordtimes.append(time.time()-self.FirstTime)
        self.vacuums.append(self.VacuumNow)
        self.currents.append(self.CurrentNow)
        
        
        self.ax2[0].clear()
        self.ax2[1].clear()
        self.ax2[1].semilogy()
        self.ax2[0].plot(self.recordtimes,self.currents,color='DarkRed')
        self.ax2[0].set_xlabel("time")
        self.ax2[0].set_ylabel("I [ A ]")
        self.ax2[1].plot(self.recordtimes,self.vacuums,color='DarkBlue')
        self.ax2[1].set_xlabel("time")
        self.ax2[1].set_ylabel("Vac [ Torr ]")
        self.ax2[1].tick_params(axis='y', which='both', labelsize=8,rotation=45)

        self.CanvasObj2.draw()
        
        
        
    def on_photo_press(self):
        return(True)
    
    def on_setcurrent_click(self):
        self.CurrentNow=float(min(float(self.CurrentVal.text()),self.HardCurrentLimit))
        self.Current.setText('%.2F' % self.CurrentNow + str(" A"))
        self.comdict['current']=self.CurrentNow
        self.TimeOfLastStep=time.time()
        return(True)
        

    

        
class DataStream(QObject):
    
    vacuumdat=pyqtSignal(float)
    finished=pyqtSignal()

    def __init__(self,comdict):
        super().__init__()
        self.vacuumlevel=1e-5
        self.comdict=comdict

    def run(self):
        while(True):
            time.sleep(5)
            self.vacuumlevel=self.vacuumlevel*1.02
            self.vacuumdat.emit(self.vacuumlevel)
            print("Sending current "+str(self.comdict['current']))

In [ ]:
BeamGui()

qt.qpa.fonts: Populating font family aliases took 144 ms. Replace uses of missing font family "Courier" with one that exists to avoid this cost. 


Sending current 1
Sending current 1
Sending current 1
Sending current 1
Sending current 1
Sending current 1
Sending current 1
Sending current 1
Sending current 1


In [ ]:
max(1,4)